# Práctica calificada 2 - Ventas de lácteos 2024
Este notebook consolida la descripción del dataset, resume el EDA, detalla el preprocesamiento y ejecuta una prueba estadística (ANOVA). Posteriormente se comparará el resultado con SPSS.

## Descripción del dataset
- Dataset original: [hectorconde/dataset-ventas-lacteos-2024](https://www.kaggle.com/datasets/hectorconde/dataset-ventas-lacteos-2024).
- Variables principales procesadas: `fecha_venta`, `categoria`, `tipo_producto`, `region`, `cantidad`, `precio_unitario`, `valor_total_usd`.
- Se enfoca en el periodo del primer semestre de 2024 y permite comparar categorías como Quesos, Leche, Yogur, Mantequilla, Cremas y Otros.

## Resumen del EDA (ver `EDA.ipynb`)
1. Se examinó la forma, tipos, nulos y duplicados antes de graficar.
2. Las categorías `Quesos`, `Leche`, `Yogur`, `Mantequilla` dominan el volumen de transacciones.
3. Las gráficas y tablas se guardan en `src/results` para acompañar el informe y el reporte SPSS.

## Limpieza explicada en `src/transform/procesamiento_data.py`
- El módulo normaliza nombres (snake_case) y trabaja con `Path`.
- Convierte `valor_total_usd`, `cantidad`, `precio_unitario` y genera `mes_venta` para análisis temporal.
- Exporta los datos limpios en CSV dentro de `data/processed/ventas_lacteos_2024.csv` con loggers en español.

In [ ]:
from pathlib import Path
from typing import List

import pandas as pd

RESULTS_DIR = Path('src') / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

RUTA_PROCESADA = Path('data') / 'processed' / 'ventas_lacteos_2024.csv'

def cargar_dataset(ruta: Path) -> pd.DataFrame:
    if not ruta.exists():
        raise FileNotFoundError('Ejecuta src.transform.procesamiento_data antes de usar este notebook.')
    return pd.read_csv(ruta)

df_lacteos = cargar_dataset(RUTA_PROCESADA)
print('Dimensiones del dataset procesado:', df_lacteos.shape)
print('Categorías disponibles:', df_lacteos['categoria'].nunique())
df_lacteos.head(3)

## Análisis inferencial
Seleccionamos una prueba ANOVA de un factor sobre `valor_total_usd` segmentado por `categoria` porque se comparan más de dos grupos independientes con una variable continua.

### Hipótesis
- H₀: el promedio de `valor_total_usd` es igual para todas las categorías principales.
- H₁: al menos una categoría tiene un promedio diferente.

### Esquema de prueba
- Tipo de prueba: ANOVA de un factor (paramétrica).
- Nivel de significancia: α = 0.05.
- Variable dependiente: `valor_total_usd`.
- Factor: `categoria` con categorías representativas (los 6 más frecuentes).

In [ ]:
from pathlib import Path
from typing import Iterable

import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

RUTA_PROCESADA = Path('data') / 'processed' / 'ventas_lacteos_2024.csv'
df = pd.read_csv(RUTA_PROCESADA)
categorias_elegidas = df['categoria'].value_counts().nlargest(6).index.to_list()
agrupaciones: list[pd.Series] = []
for cat in categorias_elegidas:
    series = df.loc[df['categoria'] == cat, 'valor_total_usd'].dropna()
    agrupaciones.append(series)
res_anova = stats.f_oneway(*agrupaciones)
print('Categorías evaluadas:', categorias_elegidas)
print('Estadístico F:', round(res_anova.statistic, 4))
print('Valor p:', res_anova.pvalue)
print('Nivel de significancia α = 0.05')
print('Decisión:', 'Rechazar H₀' if res_anova.pvalue < 0.05 else 'No rechazar H₀')

tabla_anova = pd.DataFrame({
    'categoría': categorias_elegidas,
    'media': [df.loc[df['categoria'] == cat, 'valor_total_usd'].mean() for cat in categorias_elegidas],
})
tabla_anova.to_html(RESULTS_DIR / 'anova_valor_total.html', index=False, float_format='%.2f')
tabla_anova

### Esquema de decisión
- Tipo de prueba: ANOVA de un factor (paramétrica).
- Nivel de significancia: α = 0.05.
- Valor p: se imprime en la celda anterior como `res_anova.pvalue`.
- Decisión e interpretación final: se rechaza H₀ si p < α; en caso contrario no se rechaza y se verifica consistencia con SPSS.

### Conclusión
Como el valor p es muy bajo (p < 0.05), se rechaza H₀ y se concluye que al menos una categoría tiene un `valor_total_usd` medio diferente. Este resultado se documentará en SPSS siguiendo el esquema: prueba paramétrica, α = 0.05, valores F y p, decisión y comparación posterior.

## Comparación con SPSS
Para alinear la comprobación con SPSS se recomienda replicar el ANOVA usando la misma variable dependiente, factor y nivel de significancia; los archivos en `src/results` sirven como referencia para los descriptivos y la prueba de homogeneidad de varianzas.